In [1]:
import descarteslabs as dl
from datetime import datetime
from shapely.geometry import Point
import osgeo
from osgeo import ogr, osr
from subprocess import check_output
import subprocess 

In [58]:
cmd='gsutil ls gs://dl-fires/AFMP-data/'
push=subprocess.Popen(cmd, shell=True, stdout = subprocess.PIPE)
print push.stdout

<open file '<fdopen>', mode 'rb' at 0x1097d8810>


In [8]:
cmd='gsutil ls gs://dl-fires/AFMP-data/'

process = subprocess.call('mkdir temp', shell=True)

print process

0


In [10]:
args = ['gsutil rsync gs://dl-fires/AFMP-data temp/']
p = subprocess.Popen(args, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print p.communicate()

('', 'Building synchronization state...\nStarting synchronization...\nCopying gs://dl-fires/AFMP-data/conus_fires_20180503-174713.txt...\n/ [0 files][    0.0 B/  1.5 MiB]                                                \r-\r- [1 files][  1.5 MiB/  1.5 MiB]                                                \rCopying gs://dl-fires/AFMP-data/conus_fires_20180503-175105.txt...\n- [1 files][  1.5 MiB/  3.0 MiB]                                                \r\\\r\\ [2 files][  3.0 MiB/  3.0 MiB]                                                \rCopying gs://dl-fires/AFMP-data/conus_fires_20180503-202105.txt...\n\\ [2 files][  3.0 MiB/  4.5 MiB]                                                \r|\r| [3 files][  4.5 MiB/  4.5 MiB]                                                \rCopying gs://dl-fires/AFMP-data/conus_fires_20180503-212105.txt...\n| [3 files][  4.5 MiB/  6.0 MiB]                                                \r/\r/ [4 files][  6.0 MiB/  6.0 MiB]                                     

In [46]:
def make_tile(coords): 
    # to get the tile centered around your lat/lon, give it a tilesize of 1 and a pad of the half-width of your desired image (in meters)
    bounding_box = dl.raster.dltile_from_latlon(
        lat=coords[0],
        lon=coords[1], 
        resolution=30, 
        tilesize=1, 
        pad=8400)

    return bounding_box
    

In [22]:
def search_products(date, geometry):
    print(date)
    from pprint import pprint 
    pprint(geometry['geometry']['coordinates'])
    images = dl.metadata.search(
        ['landsat:LC08:01:RT:TOAR', 'sentinel-2:L1C'],
        geom= geometry, 
        start_datetime=date, 
        end_datetime= datetime.today().isoformat()
        )
    pprint("The number of total images found is {}.".format(len(images['features'])))
    return [image['id'] for image in images['features']]

In [48]:
def download_imagery(ids, geometry):
    for image_id in ids: 
        print image_id
        dl.raster.raster(
                        image_id,
                        bands=['swir2', 'swir1', 'nir', 'alpha'],
                        scales=[[0,4000], [0, 4000], [0, 4000], None],
                        data_type='Byte',
#                         cutline=geometry,
                        save= True,
                        outfile_basename=image_id)

In [24]:
def find_imagery(fire_name, date, coords, footprint=None):
    # Where name is a str, coords is an array of [lat,long], and footprint is an optional geometry
#     if footprint:
#         image_ids = search_products(date,footprint)
#         download_imagery(image_ids, footprint)
#     else:
    dl_tile = make_tile(coords)
    image_ids = search_products(date, dl_tile)
    download_imagery(image_ids, dl_tile)

Driver Code

In [49]:
find_imagery('Test Fire', '2018-04-30',[19.36621513539742, 204.83081817626953])         
          

2018-04-30
[
  [
    [-126.6356432904732, 31.424267482726435],
    [-130.13315427145105, 29.734363666483205],
    [-124.9783633009972, 32.54120774543515],
    [-120.54258922325818, 34.62953778121783],
    ...
  ]
]
'The number of total images found is 14.'
landsat:LC08:01:RT:TOAR:meta_LC08_L1TP_042036_20180502_20180503_01_RT_v1
landsat:LC08:01:RT:TOAR:meta_LC08_L1TP_043036_20180509_20180509_01_RT_v1
sentinel-2:L1C:2018-04-30_10SFC_00_S2B_v1
sentinel-2:L1C:2018-04-30_10SFD_60_S2B_v1
sentinel-2:L1C:2018-05-02_10SEC_26_S2A_v1
sentinel-2:L1C:2018-05-02_10SFC_99_S2A_v1
sentinel-2:L1C:2018-05-02_10SFD_96_S2A_v1
sentinel-2:L1C:2018-05-02_10SGD_99_S2A_v1


KeyboardInterrupt: 